In [1]:
# autoreload modules
%load_ext autoreload
%autoreload 2

In [2]:
import bempp.api
import numpy as np

## Test Assembly

In [3]:
import MyHM.assembly as asb

In [4]:
# grid = bempp.api.shapes.sphere(h=0.2)
# grid = bempp.api.shapes.ellipsoid(r1=0.5, r2=1, r3=0.3, origin=(0, 0, 0), h=0.1)
# grid = bempp.api.shapes.ellipsoid(3, 1, 1, h=h)
# grid = bempp.api.shapes.cube(h=1)
# grid = bempp.api.shapes.cube(h=0.5)
grid = bempp.api.import_grid('grids/ribcage4-h-4.msh')
# grid = bempp.api.import_grid('grids/ribcage3-h-1.msh')

# grid.plot()
print(grid.vertices.shape)
print(grid.elements.shape)

# bempp.api.DEFAULT_DEVICE_INTERFACE = 'opencl'
bempp.api.DEFAULT_DEVICE_INTERFACE = 'numba'


(3, 1751)
(3, 3486)


In [5]:
space = bempp.api.function_space(grid, "P", 1)
k = 7
dlp = bempp.api.operators.boundary.helmholtz.double_layer(space, space, space, k)
hyp = bempp.api.operators.boundary.helmholtz.hypersingular(space, space, space, k)
boundary_operator = dlp
%time matrix_wf = boundary_operator.weak_form()
A = np.array(matrix_wf.A)

CPU times: user 58.9 s, sys: 133 ms, total: 59 s
Wall time: 16.9 s


In [6]:
# Inputs:
parameters = bempp.api.GLOBAL_PARAMETERS
device_interface = "numba"
# device_interface = "opencl"

# Block rows and columns:
cols = list(range(4,10))
rows = list(range(0,4))

# cols = list(range(grid.vertices.shape[1]))
# rows = list(range(grid.vertices.shape[1]))

# result = np.zeros((len(rows), len(cols)), dtype=np.complex128)

# Assemble dense:
%time result = asb.partial_dense_assembler(boundary_operator.descriptor, boundary_operator.domain, boundary_operator.dual_to_range, parameters, rows, cols)
%time singular_sm = asb.singular_assembler_sparse(device_interface, boundary_operator.descriptor, boundary_operator.domain, boundary_operator.dual_to_range, parameters)  

CPU times: user 5.71 s, sys: 29.6 ms, total: 5.74 s
Wall time: 5.77 s
CPU times: user 2.91 s, sys: 207 µs, total: 2.91 s
Wall time: 480 ms


In [7]:
%%time
meshgrid = np.meshgrid(rows, cols, indexing="ij")
result += singular_sm[meshgrid[0], meshgrid[1]]

CPU times: user 1.09 ms, sys: 6 µs, total: 1.09 ms
Wall time: 3.4 ms


In [8]:
# Check results:
print(np.linalg.norm(result - A[meshgrid[0], meshgrid[1]]))
print(np.linalg.norm(result - A[meshgrid[0], meshgrid[1]]) / np.linalg.norm(A[meshgrid[0], meshgrid[1]]))

6.646539573125444e-17
9.308623842080668e-17


## Test Trees

In [11]:
# Hacer min_block_size

In [9]:
import MyHM.structures as stt

In [10]:
# grid = bempp.api.shapes.sphere(h=0.2)
# grid = bempp.api.shapes.ellipsoid(r1=0.5, r2=1, r3=0.3, origin=(0, 0, 0), h=0.1)
# grid = bempp.api.shapes.ellipsoid(3, 1, 1, h=h)
# grid = bempp.api.shapes.cube(h=1)
# grid = bempp.api.shapes.cube(h=0.5)
grid = bempp.api.import_grid('grids/ribcage4-h-4.msh')
# grid = bempp.api.import_grid('grids/ribcage3-h-1.msh')

# grid.plot()
print(grid.vertices.shape)
print(grid.elements.shape)


(3, 1751)
(3, 3486)


In [11]:
bbox = grid.bounding_box
vertices = grid.vertices
space = bempp.api.function_space(grid, "P", 1)
if not space.requires_dof_transformation:
    dof_indices = list(range(vertices.shape[1]))
else:
    # TODO:
    raise NotImplementedError
    # space.dof_transformation.indices

octree = stt.Octree(vertices.T, dof_indices, bbox, max_depth=4)
%time octree.generate_tree()

tree_3d = stt.Tree3D(octree)
%time tree_3d.generate_adm_tree()

CPU times: user 14.5 ms, sys: 0 ns, total: 14.5 ms
Wall time: 14.5 ms
CPU times: user 1.52 s, sys: 10 ms, total: 1.53 s
Wall time: 1.52 s


In [12]:
tree_3d.stats

{'number_of_nodes': 44964,
 'number_of_leaves': 41652,
 'number_of_not_adm_leaves': 17630,
 'number_of_adm_leaves': 24022}

## Test compression in tree

In [17]:
import MyHM.structures as stt
from MyHM.compression.aca import ACAPP_with_assembly, ACAPP

In [31]:
grid = bempp.api.shapes.sphere(h=0.2)
# grid = bempp.api.shapes.ellipsoid(r1=0.5, r2=1, r3=0.3, origin=(0, 0, 0), h=0.1)
# grid = bempp.api.shapes.ellipsoid(3, 1, 1, h=h)
# grid = bempp.api.shapes.cube(h=1)
# grid = bempp.api.shapes.cube(h=0.5)
# grid = bempp.api.import_grid('grids/ribcage4-h-4.msh')
# grid = bempp.api.import_grid('grids/ribcage3-h-1.msh')

# grid.plot()
print(grid.vertices.shape)
print(grid.elements.shape)

# bempp.api.DEFAULT_DEVICE_INTERFACE = 'opencl'
bempp.api.DEFAULT_DEVICE_INTERFACE = 'numba'


(3, 425)
(3, 846)


In [32]:
bbox = grid.bounding_box
vertices = grid.vertices
space = bempp.api.function_space(grid, "P", 1)
if not space.requires_dof_transformation:
    dof_indices = list(range(vertices.shape[1]))
else:
    # TODO:
    raise NotImplementedError
    # space.dof_transformation.indices

octree = stt.Octree(vertices.T, dof_indices, bbox, max_depth=4)
%time octree.generate_tree()

tree_3d = stt.Tree3D(octree)
%time tree_3d.generate_adm_tree()

CPU times: user 7.45 ms, sys: 0 ns, total: 7.45 ms
Wall time: 7.06 ms
CPU times: user 1.79 s, sys: 40.7 ms, total: 1.83 s
Wall time: 1.74 s


In [35]:
tree_3d.stats

{'number_of_nodes': 48591,
 'number_of_leaves': 44321,
 'number_of_not_adm_leaves': 33377,
 'number_of_adm_leaves': 10944}

In [33]:
k = 7
dlp = bempp.api.operators.boundary.helmholtz.double_layer(space, space, space, k)
hyp = bempp.api.operators.boundary.helmholtz.hypersingular(space, space, space, k)
boundary_operator = dlp
parameters = bempp.api.GLOBAL_PARAMETERS
device_interface = "numba"

# Complete matrix:
%time matrix_wf = boundary_operator.weak_form()
A = np.array(matrix_wf.A)

CPU times: user 2.84 s, sys: 39.4 ms, total: 2.88 s
Wall time: 404 ms


In [34]:
# Compressed matrix in tree
%time tree_3d.add_compressed_matrix(ACAPP_with_assembly, device_interface, boundary_operator, parameters, epsilon=1e-3, verbose=False)

# Resultados de tiempo (ribcage4-h-4 -> 1751 vértices):
# CPU times: user 2h 51min 17s, sys: 18.7 s, total: 2h 51min 36s
# Wall time: 21min 31s

Tiempo adm: 	105.89084362983704s
Tiempo no adm: 	55.633962631225586s
CPU times: user 21min 39s, sys: 1.32 s, total: 21min 40s
Wall time: 2min 42s


In [36]:
b = np.random.rand(A.shape[1])

In [37]:
result1 = A@b

In [38]:
tree_3d.add_vector(b)

In [39]:
result2 = tree_3d.matvec_compressed(dtype=np.complex128)

In [40]:
np.linalg.norm(result1 - result2) / np.linalg.norm(result1)

0.00019681883859240122

In [41]:
# Este no llama a la parte del assembler, sino que usa la matriz entera:
%time tree_3d.add_matrix_with_ACA(A, ACAPP, epsilon=1e-3, verbose=False)

CPU times: user 1.33 s, sys: 14 µs, total: 1.33 s
Wall time: 1.33 s


In [42]:
result3 = tree_3d.matvec_compressed(dtype=np.complex128)
np.linalg.norm(result1 - result3) / np.linalg.norm(result1)
# Aunque se puede observar que ambos errores son cercanos

0.00019681883859240076

In [ ]:
# Agregar la implementación del assembly en el árbol!
# Verificar que no hayas nodos hojas admisibles con partes singulares!
# Una vez hecho lo anterior, hacer gráfico de normas de diferencias entre matriz original y comprimida a medida que
# disminuimos el valor de epsilon.